# Capter6 逻辑斯蒂回归
## 逻辑回归模型（LR）

In [1]:
import numpy as np
class LogRegress():
    def fit(self, train_x, train_y, epoch = 30, lr = 0.05):
        self.w = np.zeros(train_x.shape[1])
        self.b = 0
        self.loss_lst = []
        for i in range(epoch):
            h = 1 / (1 + np.exp(-1 * (np.dot(train_x, self.w.T) + self.b)))
            L = -1 * np.average(train_y * np.log(h) + (1 - train_y) * np.log(1 - h))
            self.loss_lst.append(L)
            dL_dw = np.average(train_x.T * (h - train_y), axis=1)
            dL_db = np.average(h - train_y)
            self.w = self.w - lr * dL_dw
            self.b = self.b - lr * dL_db
    
    def predict(self, data_x):
        pre = 1 / (1 + np.exp(-1 * (np.dot(data_x, self.w.T) + self.b)))
        pre = np.where(pre<=0.5, 0, 1)
        return pre.astype(np.int)

    def get_accuracy(self, gt_y, pre_y):
        diff = np.subtract(gt_y, pre_y)
        err = diff[diff!=0].shape[0]
        return 1 - err/gt_y.shape[0]

## 模型测试
### 准备测试样本

In [2]:
%matplotlib widget
from sklearn import datasets
import matplotlib.pyplot as plt

sample_num = 200
train_num = sample_num//2
inputs, labels = datasets.make_classification(n_samples=sample_num, n_features=2, n_classes=2, n_redundant=0)
train_x = inputs[:train_num]
train_y = labels[:train_num]
test_x = inputs[train_num:]
test_y = labels[train_num:]

### 模型训练与预测

In [3]:
model = LogRegress()
model.fit(train_x, train_y)
pre_y = model.predict(test_x)
print('accuracy: %.2f' % model.get_accuracy(test_y, pre_y))

fig, axs = plt.subplots(1, 3)
axs[0].plot(model.loss_lst)
axs[0].set_title('loss')
def plot_decision_boundary(pred_func, fea, label, ax, title):
    fea1_min, fea1_max = fea[:, 0].min() - 0.5, fea[:, 0].max() + 0.5
    fea2_min, fea2_max = fea[:, 1].min() - 0.5, fea[:, 1].max() + 0.5
    s = 0.01
    g_fea1, g_fea2 = np.meshgrid(np.arange(fea1_min, fea1_max, s), np.arange(fea2_min, fea2_max, s))
    pre_y = pred_func(np.c_[g_fea1.ravel(), g_fea2.ravel()])
    pre_y = pre_y.reshape(g_fea1.shape)
    ax.contourf(g_fea1, g_fea2, pre_y, cmap=plt.cm.Spectral)
    ax.scatter(fea[:, 0], fea[:, 1], c = label, s = 20)
    ax.set_title(title)
plot_decision_boundary(model.predict, train_x, train_y, axs[1], 'train')
plot_decision_boundary(model.predict, test_x, test_y, axs[2], 'predict')

accuracy: 0.94


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …